# Import

## Libraries

In [1]:
import os
import nest_asyncio
import warnings
import yaml
import agentops


from pathlib import Path
from dotenv import load_dotenv
from pydantic import BaseModel, Field, PositiveInt
from typing import List, Optional
# from IPython.display import display, Markdown

from crewai import Agent, Task, Crew
from crewai_tools import FileReadTool, DirectoryReadTool
from crewai.flow.flow import Flow, listen, start

load_dotenv()  # take environment variables from .env
nest_asyncio.apply()
warnings.filterwarnings("ignore")

# os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=c2494193-c82d-47a0-85e9-42017af844fb


## Conversation Transcript

In [2]:
transcript_content = Path("data/transcripts/conversation_01.txt").read_text()
print(transcript_content[:200])

CSR: Good morning, thank you for calling ShiokAh Insurance. This is Sarah speaking. How may I help you today?

Customer: Hi ah, I want to enquire about travel insurance. Planning to go Japan next mont


## Tasks & Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    "agents": "src/recsys/config/agents.yaml",
    "tasks": "src/recsys/config/tasks.yaml",
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs["agents"]
tasks_config = configs["tasks"]

In [4]:
print(
    "agents_config contains:",
    ", ".join(agents_config.keys()),
    "\ntasks_config contains:",
    ", ".join(tasks_config.keys()),
)

agents_config contains: conversation_transcript_agent, underwriting_report_agent, recommendation_agent 
tasks_config contains: process_transcript_task, generate_underwriting_report_task, give_recommendations_task


## Tools (Agent Usage)

In [5]:
directory_transcripts = DirectoryReadTool(directory="./data/transcripts")
directory_processed_policies = DirectoryReadTool(directory="./data/processed")
file_read_tool = FileReadTool()

## Pydantic Classes

In [6]:
class CustomerInfo(BaseModel):
    age: PositiveInt = Field(..., description="Age of the customer")
    medical_conditions: Optional[List[str]] = Field(
        default_factory=list, description="List of pre-existing medical conditions"
    )
    destination: str = Field(..., description="Travel destination")
    coverage_factors: List[str] = Field(
        default_factory=list, description="Factors affecting coverage"
    )
    policy_customization_factors: List[str] = Field(
        default_factory=list, description="Factors for policy customization"
    )
    coverage_priority: List[str] = Field(
        default_factory=list,
        description="Customer's priorities for coverage, e.g., destination coverage, PA benefits",
    )


class CustomerInfoList(BaseModel):
    transcript_notes: List[CustomerInfo] = Field(default_factory=list)
    underwriting_reports: List[str] = Field(default_factory=list)

# Create Crew, Agents and Tasks

In [7]:
conversation_transcript_agent = Agent(
    config=agents_config["conversation_transcript_agent"],
    tools=[directory_transcripts, file_read_tool],
    verbose=True,
)

underwriting_report_agent = Agent(
    config=agents_config["underwriting_report_agent"],
    tools=[directory_processed_policies, file_read_tool],
    verbose=True,
)

recommendation_agent = Agent(
    config=agents_config["recommendation_agent"],
    verbose=True,
)

In [8]:
process_transcript_task = Task(
    config=tasks_config["process_transcript_task"],
    agent=conversation_transcript_agent,
    output_pydantic=CustomerInfoList,
)

give_recommendations_task = Task(
    config=tasks_config["give_recommendations_task"],
    agent=recommendation_agent,
)

# generate_underwriting_report_task = Task(
#     config=tasks_config["generate_underwriting_report_task"],
#     context=[process_transcript_task],
#     agent=underwriting_report_agent,
#     async_execution=True,
# )

# Flows

In [9]:
class RecSysFlow(Flow[CustomerInfoList]):
    agentops.init()

    @start()
    async def process_transcript(self):
        print("**Processing Transcript**")
        cust_info = Crew(
            agents=[conversation_transcript_agent],
            tasks=[process_transcript_task],
            verbose=False,
        ).kickoff()
        self.state.transcript_notes = cust_info.raw
        print("**Processing Transcript ends}**")

    @listen(process_transcript)
    async def uw_report(self):
        print("**Generating UW reports**")
        reports = []

        directory_path = "./data/processed"
        files = [
            f
            for f in os.listdir(directory_path)
            if os.path.isfile(os.path.join(directory_path, f))
        ]

        if not files:
            print("No files found in the processed directory.")
            return

        for file_name in files:
            file_path = os.path.join(directory_path, file_name)

            generate_underwriting_report_task = Task(
                config=tasks_config["generate_underwriting_report_task"],
                # adding context here gives 'str' object has no attribute 'get' error
                # context=[# self.transcript_processed_customers, # f"file path: {file_path}",],
                agent=underwriting_report_agent,
                async_execution=True,
            )

            report = await Crew(
                agents=[underwriting_report_agent],
                tasks=[generate_underwriting_report_task],
                verbose=False,
            ).kickoff_async(
                inputs={
                    "cust_info": self.state.transcript_notes,
                    "file_path": file_path,
                }
            )
            reports.append(str(report))

        self.state.underwriting_reports = "\n".join(reports)
        print(f"**UW reports generated: \n{self.state.underwriting_reports}**")

    @listen(uw_report)
    def generate_recommendations(self):
        print("**Generating Recommendations**")

        recommendation = Crew(
            agents=[recommendation_agent],
            tasks=[give_recommendations_task],
            verbose=True,
        ).kickoff(
            inputs={
                "cust_info": self.state.transcript_notes,
                "uw_report": self.state.underwriting_reports,
            }
        )

        print(f"**Recommendations generated:\n{recommendation}**")
        return recommendation

    agentops.end_session("Success")

In [10]:
flow = RecSysFlow()
flow.kickoff()

**Processing Transcript**
# Agent: Conversation Transcript Analyst (Travel Insurance)
## Task: This task involves analyzing a conversation transcript between a potential customer interested in travel insurance and a customer service representative from ShiokAh Insurance. The goal is to extract key customer details from the transcript, such as age, medical conditions, destination, coverage preferences, customization requests and coverage priorities. This extracted information will be organized into a Python list and used by an underwriter to determine policy suitability.


# Agent: Conversation Transcript Analyst (Travel Insurance)
## Thought: In order to extract the key customer details from the conversation transcript, the first step is to locate the file containing the conversation.
## Using tool: List files in directory
## Tool Input: 
"{}"
## Tool Output: 
File paths: 
-./data/transcripts/conversation_01.txt


# Agent: Conversation Transcript Analyst (Travel Insurance)
## Thought: 

🖇 AgentOps: Session Stats - Duration: 12.7s | Cost: $0.011335 | LLMs: 4 | Tools: 2 | Actions: 0 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=c2494193-c82d-47a0-85e9-42017af844fb
2024-11-23 20:57:37,121 - 8525522176 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


**Processing Transcript ends}**
**Generating UW reports**
# Agent: Underwriting Report Generator (Travel Insurance)
## Task: This task requires you to generate a comprehensive underwriting report in Markdown format to assess how suitable a travel  insurance policy is for a customer's given coverage priorities mentioned in ```python
customer_details = [
    {"age": 62},
    {"medical_conditions": ["high blood pressure", "diabetes"]},
    {"destination": "Japan"},
    {"travel_dates": {"departure": "March 25", "return": "April 8"}},
    {"coverage_preferences": {
        "priority": ["medical", "baggage"],
        "medical_coverage_level": "high",
        "baggage_details": ["normal clothes", "electronics (phone, camera)"]
    }},
    {"customization_requests": {"special_activities": ["onsen (consider blood pressure)"]}},
    {"coverage_priorities": ["medical", "baggage protection", "trip cancellation"]},
    {"travel_companions": [
        {"name": "wife", "age": 60, "medical_conditions

🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().
🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()




# Agent: Underwriting Report Generator (Travel Insurance)
## Thought: To generate the report, I need to read the content of the file located at `./data/processed/income_travel_insurance_2024.txt` to understand the policy details. First, I will list the files in the directory to confirm the existence of the required file.
## Using tool: List files in directory
## Tool Input: 
"{}"
## Tool Output: 
File paths: 
-./data/processed/income_travel_insurance_2024.txt
- ./data/processed/ge_travel_insurance_2024.txt


...

🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().
🖇 AgentOps: Could not end session - no sessions detected
2024-11-23 20:58:32,911 - 8525522176 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Underwriting Report Generator (Travel Insurance)
## Final Answer: 
# GE Travel Insurance Underwriting Report

## Section 1: Customer Coverage Priorities

### Customer Coverage Needs
- **Age**: 62 (traveling with wife, age 60)
- **Medical Conditions**: High blood pressure, diabetes
- **Destination**: Japan
- **Travel Dates**: March 25 - April 8
- **Coverage Preferences**:
  - **Priority**: Medical & Baggage
  - **Medical Coverage Level**: High
  - **Baggage Details**: Normal clothes, electronics (phone, camera)
  - **Customization Requests**: Special activities include onsen (consider blood pressure)
  
### Policy Benefits Evaluation

#### Medical and Additional Expenses
- **Age Consideration**: Adults maximum 69 years old
- **Medical Coverage**: 
  - **Platinum**: 2,500,000,000 IDR
  - **Gold**: 1,000,000,000 IDR
  - **Silver**: 750,000,000 IDR
- **Medical Emergency Evacuation**: Unlimited cover (All tiers)

#### Baggage and Personal Effects
- **Reimbursement for Loss**:
  -

🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().
🖇 AgentOps: Could not end session - no sessions detected




# Agent: Travel Insurance Policy Recommendation Specialist
## Final Answer: 
# Comprehensive Travel Insurance Recommendation Report

## Customer Information

**Profile Recap:**
- **Age:** 62
- **Travel Companion:** Wife, aged 60, with no medical conditions
- **Medical Conditions:** High blood pressure and diabetes
- **Destination:** Japan
- **Travel Dates:** March 25 to April 8
- **Coverage Preferences:**
  - **Primary Priorities:** Medical and Baggage coverage
  - **Medical Coverage Level:** High
  - **Baggage Details:** Includes normal clothes and electronics (phone, camera)
  - **Special Activity Interest:** Onsen use (consider blood pressure)

## Policy Recommendation

Based on the customer's preferences and needs, the **GE Travel Insurance Platinum Plan** is the most suitable travel insurance policy. Here's a detailed decision logic:

1. **Medical Coverage:**
   - The Platinum Plan provides a substantial medical coverage of 2,500,000,000 IDR, which aligns with the customer's nee

CrewOutput(raw="# Comprehensive Travel Insurance Recommendation Report\n\n## Customer Information\n\n**Profile Recap:**\n- **Age:** 62\n- **Travel Companion:** Wife, aged 60, with no medical conditions\n- **Medical Conditions:** High blood pressure and diabetes\n- **Destination:** Japan\n- **Travel Dates:** March 25 to April 8\n- **Coverage Preferences:**\n  - **Primary Priorities:** Medical and Baggage coverage\n  - **Medical Coverage Level:** High\n  - **Baggage Details:** Includes normal clothes and electronics (phone, camera)\n  - **Special Activity Interest:** Onsen use (consider blood pressure)\n\n## Policy Recommendation\n\nBased on the customer's preferences and needs, the **GE Travel Insurance Platinum Plan** is the most suitable travel insurance policy. Here's a detailed decision logic:\n\n1. **Medical Coverage:**\n   - The Platinum Plan provides a substantial medical coverage of 2,500,000,000 IDR, which aligns with the customer's need for high-level medical coverage.\n   - I